In [ ]:
#!/usr/bin/env python
# coding: utf-8
get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold

import gc

from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings(action='ignore')


import torch
print(torch.__version__)
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR

from torchsummary import summary

from tools import EarlyStopping, eval_summary

print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
device

#### Design

In [ ]:
# from torch_design.rnn_fc import RNN_FC_Model as Model
# from torch_design.rnn_fc import RNN_FC_Dataset as Dataset
# from torch_design.rnn_fc import train_torch
# from torch_design.rnn_fc import test_torch

In [ ]:
from torch_design.basic_nn import NN_Model as Model
from torch_design.basic_nn import NN_Dataset as Dataset
from torch_design.basic_nn import train_torch
from torch_design.basic_nn import test_torch

In [ ]:
fc_cols = [
#     'tfidf_pos_word_22_0028',
#  'tfidf_pos_char_11_0000',
#  'tfidf_word_11_1263',
#  'tfidf_word_11_1516',
#  'tfidf_word_11_0552',
#  'cnt_0583',
#  'tfidf_word_22_0130',
#  'tfidf_word_11_0177',
#  'tfidf_word_11_0307',
#  'tfidf_word_22_0132',
#  'tfidf_word_11_0928',
#  'tfidf_word_11_0186',
#  'cnt_0492',
#  'tfidf_pos_word_11_0129',
#  'tfidf_char_11_0264',
#  'tfidf_pos_char_11_0650',
#  'tfidf_pos_char_11_0242',
#  'tfidf_char_11_0731',
#  'tfidf_word_11_0916',
#  'tfidf_pos_char_11_0213',
#  'tfidf_pos_word_22_0021',
#  'tfidf_char_11_0230',
#  'tfidf_pos_word_11_0391',
#  'cnt_0041',
#  'cnt_0042',
#  'tfidf_char_11_0796',
#  'tfidf_word_22_0095',
#  'tfidf_word_11_0011',
#  'tfidf_word_11_0736',
#  'tfidf_pos_char_11_0005',
#  'tfidf_pos_word_11_0077',
#  'fea__noun',
#  'cnt_0126',
#  'cnt_0223',
#  'tfidf_word_11_1439',
#  'tfidf_pos_char_11_0003',
#  'tfidf_word_11_0854',
#  'tfidf_word_11_1660',
#  'tfidf_char_11_0359',
#  'tfidf_pos_char_11_0589',
#  'cnt_0715',
#  'tfidf_pos_char_11_0415',
#  'tfidf_pos_word_11_0235',
#  'tfidf_char_11_0702',
#  'tfidf_char_11_0464',
#  'tfidf_pos_char_11_0017',
#  'tfidf_word_11_0319',
#  'tfidf_pos_char_11_0626',
#  'tfidf_pos_word_11_0420',
#  'tfidf_char_11_0657',
#  'tfidf_word_22_0091',
#  'cnt_0796',
#  'tfidf_char_11_0126',
#  'tfidf_word_11_0166',
#  'tfidf_word_33_0026',
#  'fea__text_len',
#  'tfidf_char_11_0130',
#  'tfidf_word_22_0134',
#  'cnt_0007',
#  'tfidf_pos_word_11_0044',
#  'tfidf_pos_char_11_0007',
#  'tfidf_pos_char_11_0324'
          ]

#### Load Data

In [ ]:
# merged_ts = '20191230T014439_8180'
# merged_ts = '20191229T155539'
# merged_ts = '20191231T113708_5499'
# merged_ts = '20191231T165424_6099'
# merged_ts = '20191231T162533_2022'
# merged_ts = '20200101T212111_5854_100_24161'
# merged_ts = '20200102T015155_8438_128_49980'
# merged_ts = '20200103T111811_8438_256_1774'
# merged_ts = '20200103T112827_8438_256_49980' # cv3 9458
# merged_ts = '20200104T033010_10938_512_49980'


merged_ts = '20200104T151347_9528_64_49980'

train_path = 'data/df_merged_{}_train.pkl'.format(merged_ts)
test_path = 'data/df_merged_{}_test.pkl'.format(merged_ts)

df_model = joblib.load(train_path)  
df_model = df_model.reset_index()
print('model_set\n', df_model['smishing'].value_counts())
df_test = joblib.load(test_path) 

In [ ]:
idx_cols = ['smishing', 'id', 'index']

seq_col = [c for c in df_model.columns if '_idx' in c][0]
rev_seq_col = [c for c in df_model.columns if '_ridx' in c][0]

fea_cols = [c for c in df_model.columns if c not in idx_cols + [seq_col, rev_seq_col]]

# fea_cols = [c for c in fea_cols if c in fc_cols] + [c for c in df_model.columns if 'fea__' in c]

fea_cols = list(set(fea_cols))

# fea_cols.remove(seq_col)
input_size = len(fea_cols)

vocab_size = int(merged_ts.split('_')[-1])

x_test = torch.Tensor(df_test[fea_cols].values).to(device)
x_seq_test = torch.Tensor(np.stack(df_test[seq_col].values)).long().to(device)
x_rev_seq_test = torch.Tensor(np.stack(df_test[rev_seq_col].values)).long().to(device)

In [ ]:
input_size, len(fea_cols), seq_col, rev_seq_col, vocab_size

In [ ]:
df_model[fea_cols].shape

#### Training

In [ ]:
params_dataloader = {
    'step' : 10,
    'num_workers': 2,
}

params_model = {
    'input_size':input_size, 
#     'vocab_size':vocab_size,
#     'embed_size':128, 
#     'hidden_size':16,
#     'num_layers':2,
    'device': device,
    'hidden_layers' : [4096, 2048, 1024, 1024, 512, 256]
}

print('merged_ts', merged_ts)
print(params_dataloader)
print(params_model)

[df_test.drop(c, axis=1, inplace=True) for c in df_test.columns if 'smishing_' in c]

skf = StratifiedKFold(n_splits=5, random_state=8405)

for cv, index in enumerate(skf.split(df_model[fea_cols], df_model['smishing'])):
    train_index, valid_index = index
    train_set = Dataset(df_model.loc[train_index, fea_cols + ['smishing']],
                          'smishing')#, seq_col)
    valid_set = Dataset(df_model.loc[valid_index, fea_cols + ['smishing']],
                          'smishing')#, seq_col)
    
    print(len(train_index), len(valid_index))
    print('\nCV', cv)
    model = Model(**params_model).to(device)
    
    epoch = 1
    if cv == 0:
#         print(summary(model, (input_size, )))
        print(model.train())
    
    early_stopping = EarlyStopping(patience=10, min_epoch=100, verbose=True)
    
    pos_weight = torch.Tensor([1., 10.,])
#     pos_weight = torch.Tensor([1., 1.,])
    criterion = torch.nn.BCEWithLogitsLoss(reduction='sum', pos_weight=pos_weight).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    scheduler = StepLR(optimizer, step_size=10, gamma=0.9)

    model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
    print('model_ts', model_ts)
    print('Epoch:', epoch)

    N_EPOCHS = 100
    for e in tqdm_notebook(range(epoch, epoch + N_EPOCHS), total=N_EPOCHS, desc = 'CV {} Epoch'.format(cv)):
        train_loss, train_acc = train_torch(model, train_set, criterion, optimizer, scheduler, device, 
                                            **params_dataloader)
        valid_loss, valid_acc, y_true, y_score = test_torch(model, valid_set, criterion, device,
                                                            **params_dataloader)
        print('[{}] CV {} Epoch {}\n\tTrain loss: {}\tValid loss: {}\t{}\n\t Train Acc: {}\t Valid Acc:{}'.format(
            datetime.now().strftime('%Y%m%dT%H%M%S'), 
            cv, e, train_loss, valid_loss, train_loss / valid_loss , train_acc, valid_acc))
        
        eval_dict = eval_summary(y_true, y_score, cut_off=0.5)
        print('\t', eval_dict)
        
        early_stopping(-eval_dict['auc'], model)

        if early_stopping.early_stop:
            print("\tEarly stopping epoch {}, valid loss {}".format(e, valid_loss))
            break
    
        epoch = e + 1
    
    # load the last checkpoint with the best model
    model.load_state_dict(torch.load('checkpoint.pt'))
    torch.save(model.state_dict(), 'model/{}_{}_{}.model'.format(model_ts, cv, early_stopping.best_epoch))
    print('\nBest_Epoch', early_stopping.best_epoch, 'auc', early_stopping.best_score)
    
    valid_loss, valid_acc, y_true, y_score = test_torch(model, valid_set, criterion, device, **params_dataloader)
    valid_dict = eval_summary(y_true, y_score, cut_off=0.5)
    print('END<valid> CV {} eval summary\n'.format(cv), valid_dict)
    
    train_loss, train_acc, y_true, y_score = test_torch(model, train_set, criterion, device, **params_dataloader)
    train_dict = eval_summary(y_true, y_score, cut_off=0.5)
    print('END<train> CV {} eval summary\n'.format(cv), train_dict)
    
    print('train_auc - valid_auc:', train_dict['auc'] - valid_dict['auc'])
    
    model.eval()
    pred_col = 'smishing_{}'.format(cv)
    df_test[pred_col] = torch.sigmoid(model(x_test))[:, 1].cpu().detach().numpy()
    df_test[[pred_col]].to_csv('submit/{}_{}_nn.csv'.format(model_ts, pred_col), index=True)
    
    del train_set, valid_set
    
#     break

#### Predict Train

In [ ]:
df = pd.Series(y_score)
df.hist(bins=100, figsize=(20, 5))
(df * 10).astype(int).value_counts(sort=False)

In [ ]:
# df_model[(y_score <= 0.5) & (y_true == 1)]['text']

In [ ]:
# df_model[(y_score > 0.5) & (y_true == 0)]['text']

#### Predict Test

In [ ]:
pred_cols = [c for c in df_test.columns if 'smishing_' in c]
print(len(pred_cols))
df_test['pred_max'] = df_test[pred_cols].max(axis=1)
df_test['pred_min'] = df_test[pred_cols].min(axis=1)
df_test['pred_mean'] = df_test[pred_cols].mean(axis=1)
df_test['pred_std'] = df_test[pred_cols].std(axis=1)

print(df_test['pred_std'].max(), df_test['pred_std'].min(), df_test['pred_std'].mean())

df_test['smishing'] = df_test['pred_mean']

In [ ]:
df_test['smishing'].hist(bins=100, figsize=(20, 5))

In [ ]:
for c in pred_cols:
    print(c)
    print((df_test[c] * 10).astype(int).value_counts(sort=False))

In [ ]:
(df_test['smishing'] * 10).astype(int).value_counts(sort=False)

In [ ]:
model_ts

In [ ]:
df_test[['smishing']].to_csv('submit/{}_nn.csv'.format(model_ts), index=True)
# df_test[['id', 'smishing', 'text']].sort_values('smishing', ascending=False).to_csv('{}_text.csv'.format(model_ts), index=False)
